### **Import libraries and load datasets.**

* Import Python libraries
* Load messages.csv into a dataframe and inspect the first few lines.
* Load categories.csv into a dataframe and inspect the first few lines.

In [1]:
# import libraries
import pandas as pd
from sqlalchemy import create_engine

In [ ]:
# load messages dataset
messages = pd.read_csv('messages.csv')
messages.head()

In [ ]:
# load categories dataset
categories = pd.read_csv('categories.csv')
categories.head()

### **Merge datasets.**

* Merge the messages and categories datasets using the common id
* Assign this combined dataset to df, which will be cleaned in the following steps

In [ ]:
# merge datasets
df = messages.merge(categories, how='outer',on=['id'])
df.head()

### **Split categories into separate category columns.**

* Split the values in the categories column on the ; character so that each value becomes a separate column. You'll find this method very helpful! Make sure to set expand=True.
* Use the first row of categories dataframe to create column names for the categories data.
* Rename columns of categories with new column names.

In [ ]:
detail_categories = categories.categories.str.split(';', expand=True)

new_header = detail_categories.iloc[0].str.split('-').str.get(0) #grab the first row for the header
#detail_categories = detail_categories[1:] #take the data less the header row
detail_categories.columns = new_header #set the header row as the df header

for column in detail_categories:
    # set each value to be the last character of the string
    detail_categories[column] = detail_categories[column].str.split('-').str.get(1)
    
    # convert column from string to numeric
    detail_categories[column] = detail_categories[column].astype(int)

detail_categories

In [ ]:
# drop the original categories column from `df`
df.drop(['categories'], axis=1, inplace=True)
df

In [ ]:
# concatenate the original dataframe with the new `categories` dataframe
df = pd.concat([df, detail_categories], axis=1, join='inner')
df

### **Remove duplicates.**

* Check how many duplicates are in this dataset.
* Drop the duplicates.
* Confirm duplicates were removed.

In [ ]:
cols=['id', 'message', 'original', 'genre', 'related', 'request', 'offer',
       'aid_related', 'medical_help', 'medical_products', 'search_and_rescue',
       'security', 'military', 'child_alone', 'water', 'food', 'shelter',
       'clothing', 'money', 'missing_people', 'refugees', 'death', 'other_aid',
       'infrastructure_related', 'transport', 'buildings', 'electricity',
       'tools', 'hospitals', 'shops', 'aid_centers', 'other_infrastructure',
       'weather_related', 'floods', 'storm', 'fire', 'earthquake', 'cold',
       'other_weather', 'direct_report', 'related', 'request', 'offer',
       'aid_related', 'medical_help', 'medical_products', 'search_and_rescue',
       'security', 'military', 'child_alone', 'water', 'food', 'shelter',
       'clothing', 'money', 'missing_people', 'refugees', 'death', 'other_aid',
       'infrastructure_related', 'transport', 'buildings', 'electricity',
       'tools', 'hospitals', 'shops', 'aid_centers', 'other_infrastructure',
       'weather_related', 'floods', 'storm', 'fire', 'earthquake', 'cold',
       'other_weather', 'direct_report']

In [ ]:
# check number of duplicates 
df.duplicated(subset=cols, keep='first').sum()

In [ ]:
# drop duplicates
df.drop_duplicates()

In [ ]:
# check number of duplicates
# 26386 rows before and 26345 after so 41 dups, looks, good

### **Save the clean dataset into an sqlite database.**

You can do this with pandas to_sql method combined with the SQLAlchemy library. Remember to import SQLAlchemy's create_engine in the first cell of this notebook to use it below.

In [ ]:
engine = create_engine('sqlite:///InsertDatabaseName.db')
df.to_sql('Messages', engine, index=False)

### **Use this notebook to complete etl_pipeline.py**